In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import urllib.request as request
import io
from sklearn.metrics import confusion_matrix

%matplotlib inline

# Classificação Multinomial e o reconhecimento de caracteres manuscritos.

Reproduzido do material de  R. Jordan Crouser do Smith College (2016)

A classificação multinomial extende o classificador SVM para situações em que mais de uma categoria de objetos aparece.

Vamos usar um classificador destes para reconhecimento de caracteres manuscritos.
Os dados serão do [Optical Recognition of Handwritten Digits Data Set](http://archive.ics.uci.edu/ml/datasets/optical+recognition+of+handwritten+digits) que contém imagens de dígitos de 0 a 9 manuscritos por voluntários.

O código abaixo recupera os dados, já divididos em treinamento e validação

In [2]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra -O optdigits.tra
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes -O optdigits.tes

--2025-02-02 22:17:31--  http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘optdigits.tra’

optdigits.tra           [  <=>               ] 550.43K  1.43MB/s    in 0.4s    

2025-02-02 22:17:31 (1.43 MB/s) - ‘optdigits.tra’ saved [563639]

--2025-02-02 22:17:31--  http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘optdigits.tes’

optdigits.tes           [  <=>               ] 258.51K  1.02MB/s    in 0.2s    

2025-02-02 22:17:32 (1.02 MB/s) - ‘optdigits.tes’ sa

O código a seguir coloca os dados em dataframes.

In [3]:
X_train = pd.read_csv('optdigits.tra', header=None)
y_train = X_train[64]
X_train = X_train.drop(X_train.columns[64], axis=1)

X_test = pd.read_csv('optdigits.tes', header=None)
y_test = X_test[64]
X_test = X_test.drop(X_test.columns[64], axis=1)

Podemos construir um classificador SVM linear multimodal para esta base:

In [4]:
svc = SVC(kernel='linear', C=1)
svc.fit(X_train, y_train)
# Mostra a matriz de confusão dos dados de treinamento
cm = confusion_matrix(y_train, svc.predict(X_train))
cm_df = pd.DataFrame(cm.T, index=svc.classes_, columns=svc.classes_)
print("Matriz de confusão com dados de treinamento")
print(cm_df)

Matriz de confusão com dados de treinamento
     0    1    2    3    4    5    6    7    8    9
0  376    0    0    0    0    0    0    0    0    0
1    0  389    0    0    0    0    0    0    0    0
2    0    0  380    0    0    0    0    0    0    0
3    0    0    0  389    0    0    0    0    0    0
4    0    0    0    0  387    0    0    0    0    0
5    0    0    0    0    0  376    0    0    0    0
6    0    0    0    0    0    0  377    0    0    0
7    0    0    0    0    0    0    0  387    0    0
8    0    0    0    0    0    0    0    0  380    0
9    0    0    0    0    0    0    0    0    0  382


In [5]:
cm = confusion_matrix(y_test, svc.predict(X_test))
cm_df = pd.DataFrame(cm.T, index=svc.classes_, columns=svc.classes_)
print("Matriz de confusão com dados de teste")
print(cm_df)

Matriz de confusão com dados de teste
     0    1    2    3    4    5    6    7    8    9
0  177    0    0    1    0    0    0    0    0    1
1    0  178    7    0    0    0    0    0    9    0
2    0    0  170    5    0    1    0    0    1    0
3    0    0    0  171    0    0    0    0    5    4
4    0    0    0    0  180    0    1    1    0    1
5    1    0    0    2    0  180    0    7    2    3
6    0    3    0    0    0    0  179    0    0    0
7    0    0    0    2    0    0    0  165    0    0
8    0    1    0    1    1    0    1    0  157    1
9    0    0    0    1    0    1    0    6    0  170


Por meio da matriz de confusão dos dados de teste, podem ser calculadas as métricas:


*   Sensibilidade: $VP / (VP + FN)$
*   Especificidade: $VN / (FP+VN)$
*   Acurácia: $(VP+VN)/N$
*   Precisão: $VP/(VP+FP)$
*   F-Score: $(2*P*S)/(P+S)$



In [6]:
VP = 0
FN = 0
VN = 0
FP = 0
N = 0

for i in range(10):
  for j in range(10):
    if i == j:
      VP += cm[i][j]
    else:
      if i < j:
        FP += cm[i][j]
      else:
        FN += cm[i][j]
    N += cm[i][i]
sensibilidade = VP/(VP+FN)
especificidade = VN/(FP+VN)
acuracia = (VP+VN)/N
precisao = VP/(VP+FP)
f_score = (2*sensibilidade*precisao)/(sensibilidade+precisao)

print("Sensibilidade: ",sensibilidade)
print("Especificidade: ",especificidade)
print("Acurácia: ",acuracia)
print("Precisão: ",precisao)
print("F-Score: ",f_score)

Sensibilidade:  0.971862689926843
Especificidade:  0.0
Acurácia:  0.1
Precisão:  0.9885518030910132
F-Score:  0.9801362088535754
